In [1]:
import pandas as pd
import numpy as np
from sklearn import neighbors, linear_model, model_selection, metrics
import seaborn as sns

In [2]:
from sklearn import neighbors
from sklearn.metrics import mean_squared_error
from math import sqrt

In [6]:
df = pd.read_csv('preprocessed_data_craigslist.csv').drop(
    ['Unnamed: 0'], axis=1).drop_duplicates(subset=None, keep='first', inplace=False)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2736 entries, 0 to 4027
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   price                 2736 non-null   float64
 1   numimage              2736 non-null   float64
 2   condition             1662 non-null   float64
 3   bicycleframematerial  1372 non-null   object 
 4   bicycletype           1710 non-null   object 
 5   braketype             537 non-null    object 
 6   electricassist        2736 non-null   int64  
 7   framesize             1142 non-null   float64
 8   handlebartype         352 non-null    object 
 9   suspension            2736 non-null   int64  
 10  wheelsize             1262 non-null   float64
 11  sizedimensions        2736 non-null   int64  
dtypes: float64(5), int64(3), object(4)
memory usage: 277.9+ KB


In [7]:
df = df[(10 < df.price) & (df.price < 1000)
        ].reset_index().drop(['index'], axis=1)
df

,price,numimage,condition,bicycleframematerial,bicycletype,braketype,electricassist,framesize,handlebartype,suspension,wheelsize,sizedimensions
0,30.0,6.0,4.0,NaN,NaN,NaN,0,NaN,NaN,0,NaN,0
1,112.0,4.0,4.0,NaN,track,NaN,0,NaN,NaN,0,NaN,0
2,167.0,7.0,4.0,NaN,track,NaN,0,NaN,NaN,0,NaN,0
3,125.0,9.0,3.0,aluminum,mountain,special,0,2.0,special,1,26.0,0
4,75.0,0.0,1.0,aluminum,mountain,mechanical,0,NaN,NaN,0,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...
2154,20.0,20.0,NaN,steel,kids,NaN,0,2.0,NaN,0,12.0,0
2155,75.0,2.0,NaN,NaN,NaN,NaN,0,3.0,NaN,0,26.0,0
2156,100.0,2.0,2.0,aluminum,mountain,mechanical,0,3.0,NaN,0,24.0,0
2157,110.0,10.0,3.0,NaN,NaN,NaN,0,NaN,NaN,0,NaN,0


In [8]:
pd.get_dummies(df).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2159 entries, 0 to 2158
Data columns (total 33 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   price                              2159 non-null   float64
 1   numimage                           2159 non-null   float64
 2   condition                          1333 non-null   float64
 3   electricassist                     2159 non-null   int64  
 4   framesize                          963 non-null    float64
 5   suspension                         2159 non-null   int64  
 6   wheelsize                          1050 non-null   float64
 7   sizedimensions                     2159 non-null   int64  
 8   bicycleframematerial_alloy         2159 non-null   uint8  
 9   bicycleframematerial_aluminum      2159 non-null   uint8  
 10  bicycleframematerial_carbon fiber  2159 non-null   uint8  
 11  bicycleframematerial_composite     2159 non-null   uint8

In [9]:
X = pd.get_dummies(df.drop(columns=['price']))
display(X)
y = df.price

X_train, X_test, y_train, y_test = model_selection.train_test_split(
    X, y, test_size=0.25,random_state=52)

,numimage,condition,electricassist,framesize,suspension,wheelsize,sizedimensions,bicycleframematerial_alloy,bicycleframematerial_aluminum,bicycleframematerial_carbon fiber,...,bicycletype_mountain,bicycletype_recumbent/trike,bicycletype_road,bicycletype_tandem,bicycletype_track,bicycletype_unicycle,braketype_hydraulic,braketype_mechanical,braketype_special,handlebartype_special
0,6.0,4.0,0,NaN,0,NaN,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,4.0,4.0,0,NaN,0,NaN,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,7.0,4.0,0,NaN,0,NaN,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,9.0,3.0,0,2.0,1,26.0,0,0,1,0,...,1,0,0,0,0,0,0,0,1,1
4,0.0,1.0,0,NaN,0,NaN,0,0,1,0,...,1,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2154,20.0,NaN,0,2.0,0,12.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2155,2.0,NaN,0,3.0,0,26.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2156,2.0,2.0,0,3.0,0,24.0,0,0,1,0,...,1,0,0,0,0,0,0,1,0,0
2157,10.0,3.0,0,NaN,0,NaN,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Preprocessing (tentative de scaling)

In [10]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1)).fit(X_train)

x_train_scaled = scaler.transform(X_train)
X_train = pd.DataFrame(x_train_scaled, columns=X_train.columns)

x_test_scaled = scaler.transform(X_test)
X_test = pd.DataFrame(x_test_scaled, columns=X_test.columns)

X_train, X_test

(      numimage  condition  electricassist  framesize  suspension  wheelsize  \
 0     0.250000   1.000000             0.0        NaN         0.0        NaN   
 1     0.166667   0.666667             0.0        0.5         0.0   0.789474   
 2     0.125000   1.000000             0.0        0.5         0.0   0.315789   
 3     0.416667   1.000000             1.0        0.5         0.0   0.842105   
 4     0.291667   1.000000             0.0        NaN         0.0        NaN   
 ...        ...        ...             ...        ...         ...        ...   
 1614  0.000000        NaN             0.0        NaN         0.0        NaN   
 1615  0.083333   1.000000             0.0        1.0         0.0   0.105263   
 1616  0.000000        NaN             0.0        NaN         0.0        NaN   
 1617  0.250000   1.000000             0.0        NaN         0.0        NaN   
 1618  0.375000   0.666667             0.0        0.5         0.0   1.000000   
 
       sizedimensions  bicycleframemat

# Linear Regression

In [11]:
linear_regression = linear_model.LinearRegression()

linear_regression.fit(X_train, y_train)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [ ]:
linear_regression.intercept_

In [ ]:
linear_regression.coef_

In [ ]:
y_pred = linear_regression.predict(X_test)
print('Coefficients: \n', linear_regression.coef_, len(linear_regression.coef_))
# The mean squared error
print("RMSE: %.2f" %
      sqrt(np.mean((linear_regression.predict(X_test) - y_test) ** 2)))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % linear_regression.score(X_test, y_test))

In [ ]:
df_pred = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
df_pred = df_pred[(df_pred['Predicted'] > 20) & (df_pred['Predicted'] < 2000)]
df_pred

In [ ]:
from matplotlib import pyplot as plt

plt.figure(figsize=(12, 10))
plt.scatter(df_pred['Actual'], df_pred['Predicted'])
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)

plt.xlabel('Actual price', fontsize=20)
plt.ylabel('Predicted price', fontsize=20)

sns.despine()

In [ ]:
df['price'].hist(bins=50)

In [ ]:
#import statsmodels.api as sm
# X_train = sm.add_constant(X_train) # adding a constant
# X_test = sm.add_constant(X_test) # adding a constant
#
#
#model = sm.OLS(y_train, X_train).fit()
#predictions = model.predict(X_test)
#
#print_model = model.summary()
# print(print_model)

# KNN

In [ ]:
rmse_val = []  # to store rmse values for different k
for K in range(40):
    K = K+1
    model = neighbors.KNeighborsRegressor(n_neighbors=K)
    model.fit(X_train, y_train)  # fit the model
    pred = model.predict(X_test)  # make prediction on test set
    error = sqrt(mean_squared_error(y_test, pred))  # calculate rmse
    rmse_val.append(error)  # store rmse values
fit_df = pd.DataFrame({'k': range(1, 41), 'RMSE': rmse_val}).set_index('k')
display(fit_df)
best_k = int(fit_df.idxmin())
best_k

In [ ]:
fit_df.plot(ylabel='RMSE', figsize=[12, 8], fontsize=20)
plt.plot([best_k, best_k], [fit_df['RMSE'].min()-5, fit_df['RMSE'].max()+5], color='red',
         linestyle='dashed', linewidth=2)

sns.despine()

In [ ]:
model = neighbors.KNeighborsRegressor(n_neighbors=best_k)
model.fit(X_train, y_train)  # fit the model
pred_KNN = model.predict(X_test)

In [ ]:
df_pred_KNN = pd.DataFrame({'Actual': y_test, 'Predicted': pred_KNN})
df_pred_KNN = df_pred_KNN[(df_pred_KNN['Predicted'] > 20) & (
    df_pred_KNN['Predicted'] < 2000)]
df_pred_KNN

In [ ]:
# Linear Regression vs KNN
print('RMSE for linear regression : ', sqrt(
    mean_squared_error(y_test, y_pred)))
print('RMSE for KNN with best K : ', sqrt(
    mean_squared_error(y_test, pred_KNN)))

# Random Forest Regression

In [ ]:
#from sklearn.model_selection import GridSearchCV
#
# grid = {
#    'n_estimators': [200,300,400,500],
#    'max_features': ['sqrt','log2'],
#    'max_depth' : [3,4,5,6,7],
#    'random_state' : [52]
# }
# Grid Search function
#CV_rfr = GridSearchCV(estimator=RandomForestRegressor(), param_grid=grid, cv= 5)
#CV_rfr.fit(X_train, y_train)
#

In [ ]:
# CV_rfr.best_params_

# Output is :

'''
{'max_depth': 7,
 'max_features': 'sqrt',
 'n_estimators': 500,
 'random_state': 52}
 '''

In [ ]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators=500, max_features='sqrt',
                           max_depth=7, random_state=52).fit(X_train, y_train)
pred_RF = rf.predict(X_test)
mse = mean_squared_error(y_test, pred_RF)
rmse = mse**.5
print(rmse)

In [ ]:
df_pred_RF = pd.DataFrame({'Actual': y_test, 'Predicted': pred_RF})
df_pred_RF = df_pred_RF[(df_pred_RF['Predicted'] > 20)
                        & (df_pred_RF['Predicted'] < 2000)]
df_pred_RF

In [ ]:
plt.figure(figsize=(12,12))
plt.bar(X_train.columns, rf.feature_importances_)
plt.xticks(rotation=45)

# Sum-up

In [ ]:
# Linear Regression vs KNN vs RF
print('RMSE for linear regression : ', sqrt(
    mean_squared_error(y_test, y_pred)))
print('RMSE for KNN with best K : ', sqrt(
    mean_squared_error(y_test, pred_KNN)))
print('RMSE for RF with best params : ', sqrt(
    mean_squared_error(y_test, pred_RF)))


In [ ]:
df_pred_sum_up = pd.DataFrame({'Actual': y_test,'Predicted with Linear Regression':y_pred,'Predicted with KNN k ='+str(best_k):pred_KNN,'Predicted with Random Forest': pred_RF})
average_pred = df_pred_sum_up.drop("Actual",axis=1).mean(axis=1)
df_pred_sum_up['Average']=average_pred

df_pred_sum_up = df_pred_sum_up[(df_pred_sum_up['Predicted with Linear Regression'] > 20)
                        & (df_pred_sum_up['Predicted with Linear Regression'] < 2000)]
df_pred_sum_up = df_pred_sum_up[(df_pred_sum_up['Predicted with KNN k ='+str(best_k)] > 20)
                        & (df_pred_sum_up['Predicted with KNN k ='+str(best_k)] < 2000)]
df_pred_sum_up = df_pred_sum_up[(df_pred_sum_up['Predicted with Random Forest'] > 20)
                        & (df_pred_sum_up['Predicted with Random Forest'] < 2000)]

df_pred_sum_up.reset_index().drop('index',axis=1)

In [ ]:
print('RMSE for Average : ', sqrt(
    mean_squared_error(y_test, average_pred)))

In [ ]:
x = df_pred_sum_up.index

fig = plt.figure(figsize=(8,6))  #setting the figure size
plt.scatter(x, df_pred_sum_up['Predicted with Linear Regression'],marker='.',color='r') 

plt.scatter(x, df_pred_sum_up['Predicted with KNN k ='+str(best_k)],marker='.',color='g') 
plt.scatter(x, df_pred_sum_up['Predicted with Linear Regression'],marker='.',color='r') 
plt.scatter(x, df_pred_sum_up['Predicted with Random Forest'],marker='.',color='b') 
plt.scatter(x,df_pred_sum_up['Average'],marker='x')

plt.show()